# Working with Text

In [ ]:
%matplotlib inline

Libraries for numerics

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats

Libraries for plotting

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

Libraries for string manipulation

In [ ]:
import string
import re

Libraries for functional programming

In [ ]:
from functools import reduce, partial
import itertools as it
import operator as op
import toolz as tz
import toolz.curried as c

## String methods

In [ ]:
s = "  Avoid taking unnecessary gambles. Lucky numbers: 12, 15, 23, 28, 37\n"

### Removing leading and trailing whitespace

In [ ]:
s.strip()

In [ ]:
s.lstrip()

In [ ]:
s.rstrip()

In [ ]:
s = s.strip()

### Changing case

In [ ]:
s.lower()

In [ ]:
s.upper()

In [ ]:
s.title()

### Checking conditions

In [ ]:
s.startswith('Avoid')

In [ ]:
s.endswith('37')

In [ ]:
s.isalpha()

In [ ]:
s.isnumeric()

In [ ]:
s.isspace()

In [ ]:
s.isprintable()

### Counting and indexing

In [ ]:
s.count('a')

In [ ]:
s.count('gambles')

In [ ]:
s.find('gambles')

In [ ]:
s[27:]

In [ ]:
s.find('foobar')

In [ ]:
s.index('gambles')

In [ ]:
try:
    s.index('foobar')
except ValueError as e:
    print(e)

### Splitting and joining

In [ ]:
s.split()

In [ ]:
s.split(':')

In [ ]:
'-'.join(s.split())

### Replacing

In [ ]:
s.replace('gambles', 'risk')

### Translating

In [ ]:
table = str.maketrans(string.ascii_lowercase, string.ascii_uppercase, string.punctuation)
s.translate(table)

In [ ]:
table = str.maketrans('', '', string.punctuation)
s.translate(table)

**Exercise: Caesar Cipher**

A Caesar cipher with offset $k$ converts a character into the character $k$ letters down, looping around if this goes past `z`. Non-characters (numbers, spaces, punctuation) are left intact. For instance, with offset=3, we get `abcXYZ` being coded as `defABC`. Write an function `encode(k, s)` where `k` is the offset and `s` the string to be coded.  Write a `decode(k, s)` function that decodes encrypted ciphers. Test it out on the fortune. 

In [ ]:
def encode(k, s):    
    table = str.maketrans(
        string.ascii_lowercase + string.ascii_uppercase,
        string.ascii_lowercase[k:] + string.ascii_lowercase[:k] + 
        string.ascii_uppercase[k:] + string.ascii_uppercase[:k])
    return s.translate(table)

In [ ]:
encode(3, 'abcXYZ')

In [ ]:
def decode(k, s):
    return encode(-k, s)

In [ ]:
code = encode(3, s)

In [ ]:
code

In [ ]:
decode(3, code)

## Counting words

To count words, we typically do the following preprocessing:
    
- Convert to lower (or upper) case
- Remove punctuation
- Split on blank space
- Count each word in list

In [ ]:
s

### Preprocessing

In [ ]:
words = s.lower().translate(str.maketrans('','',string.punctuation)).split()

### Using a Counter (bag)

In [ ]:
from collections import Counter

In [ ]:
Counter(words)

### Using a dictionary

In [ ]:
counter = {}
for word in words:
    counter[word] = counter.get(word, 0) + 1

In [ ]:
counter

### Using a `defaultdict`

In [ ]:
from collections import defaultdict

In [ ]:
d = defaultdict(int)

In [ ]:
for word in words:
    d[word] += 1

In [ ]:
d

### Using a functional pipe

In [ ]:
tz.pipe(
    s,
    lambda s: s.lower(),
    lambda s: s.translate(str.maketrans('', '', string.punctuation)),
    lambda s: s.split(),
    tz.frequencies
)

### Modification for collection of strings

In [ ]:
ss = [s, s, s]

In [ ]:
ss

In [ ]:
tz.pipe(
    ss,
    c.map(lambda s: s.lower()),
    c.map(lambda s: s.translate(str.maketrans('', '', string.punctuation))),
    c.mapcat(lambda s: s.split()),
    tz.frequencies
)

## String to vector

To analyze text, we typically need to convert it to a vector format. There are several ways to do so. Here we show the most obvious method known as one-hot encoding.

### One hot character encoding

We first encode the string 'abcabc' as the vector [0,1,2,0,1,2]. For one-hot encoding, we next convert this to the one-hot encoded matrix

```python
array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])
```

In [ ]:
idx = 0
index = {}
for ch in s:
    if not ch in index:
        index[ch] = idx
        idx += 1

In [ ]:
index

#### Categorical encoding

In [ ]:
nchars = len(index)

In [ ]:
vs = np.array([index[ch] for ch in s])

In [ ]:
vs

#### One-hot encoding

In [ ]:
n = len(vs)
p = len(index)
m = np.zeros((n,p), dtype='int')
i = np.arange(len(vs))
m[i, vs] = 1
m

#### Reverse index lookup

In [ ]:
reverse_index = dict(zip(index.values(), index.keys()))

In [ ]:
''.join(reverse_index[v] for v in vs)

### One hot encoding for words.

In [ ]:
words = ' '.join([s,s]).lower().translate(str.maketrans('', '', string.punctuation)).split()

In [ ]:
pos = 0
index = {}
for word in words:
    if word not in index:
        index[word] = pos
        pos += 1

#### Categorical encoding

In [ ]:
ws = np.array([index[word] for word in words])

In [ ]:
ws

#### One-hot encoding

In [ ]:
n = len(ws)
p = len(index)
m = np.zeros((n,p), dtype='int')
i = np.arange(len(ws))
m[i, ws] = 1
m

#### Reverse lookup

In [ ]:
reverse_index = dict(zip(index.values(), index.keys()))

In [ ]:
' '.join(reverse_index[w] for w in ws)

## Regular expressions

In [ ]:
s

### Literal match

In [ ]:
re.findall(r'gambles', s)

### Quantifiers `.`, `{m,n}`, `+`, `*`

In [ ]:
re.findall(r'gam.les', s)

In [ ]:
re.findall(r'g.*s', s)

### Non-greedy quantifier.

In [ ]:
re.findall(r'g.*?s', s)

### Special characters

In [ ]:
re.findall(r'\bg.*?s\b', s)

In [ ]:
re.findall(r'\b\w+?\b', s)

In [ ]:
re.findall(r'\b\d+?\b', s)

In [ ]:
re.findall(r'\b[a-zA-Z]+?\b', s)

### Begin and end anchors

In [ ]:
re.findall(r'\w+', s)

In [ ]:
re.findall(r'^\w+', s)

In [ ]:
re.findall(r'\w+$', s)

### Capture groups

In [ ]:
pat = r'\b(\d)(\d)?\b'

In [ ]:
re.findall(pat, s)

### Using search and match objects

In [ ]:
re.search(pat, s)

In [ ]:
m = re.search(pat, s)

In [ ]:
m.string

In [ ]:
m.group()

In [ ]:
m.groups()

### Replacement using capture groups

In [ ]:
rep = r'\2\1'
re.sub(pat, rep, s)

### Using compiled patterns

In [ ]:
pat = re.compile(r'\b[a-zA-Z]+?\b')
pat.findall(s)